# experiment_5_future_engineering.ipynb

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 95.5 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle

In [3]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 464MB/s]


In [6]:
! unzip walmart-recruiting-store-sales-forecasting.zip

Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Install required packages
import subprocess
import sys

In [8]:
!pip install prophet plotly mlflow dagshub xgboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import mlflow
import mlflow.sklearn
import dagshub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [64]:
# ================================================================================
# STEP 1: SETUP MLflow AND DAGSHUB
# ================================================================================

def setup_mlflow():
    """Setup MLflow and DagsHub tracking"""
    print("🔧 Setting up MLflow and DagsHub...")

    # End any active runs first
    try:
        mlflow.end_run()
    except:
        pass

    # Initialize DagsHub
    try:
        dagshub.init(
            repo_owner='konstantine25b',
            repo_name='Walmart-Recruiting---Store-Sales-Forecasting',
            mlflow=True
        )
        print("✅ DagsHub initialized successfully!")
    except Exception as e:
        print(f"⚠️ DagsHub init warning: {e}")

    # Set MLflow tracking URI
    mlflow.set_tracking_uri("https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow")

    # Create unique experiment name with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_name = f"Experiment_5_Feature_Engineering_Pipeline_{timestamp}"

    try:
        experiment_id = mlflow.create_experiment(experiment_name)
        print(f"✅ Created new experiment: {experiment_name}")
    except mlflow.exceptions.MlflowException as e:
        if "already exists" in str(e):
            experiment = mlflow.get_experiment_by_name(experiment_name)
            experiment_id = experiment.experiment_id
            print(f"✅ Using existing experiment: {experiment_name}")
        else:
            # Fallback to default experiment
            experiment_name = "Default"
            mlflow.set_experiment(experiment_name)
            print(f"⚠️ Using default experiment due to: {e}")

    mlflow.set_experiment(experiment_name)

    print(f"✅ MLflow setup complete!")
    print(f"🔗 Tracking URI: {mlflow.get_tracking_uri()}")
    print(f"📊 Experiment: {experiment_name}")

    return experiment_name

# ================================================================================
# STEP 2: DATA LOADING
# ================================================================================

def load_raw_data():
    """Load and merge all datasets"""
    with mlflow.start_run(run_name="Data_Loading_Complete", nested=True) as run:
        print("📂 Loading datasets...")

        try:
            # Load main datasets
            train_df = pd.read_csv('train.csv')
            stores_df = pd.read_csv('stores.csv')
            features_df = pd.read_csv('features.csv')

            print(f"✅ Loaded datasets:")
            print(f"   📊 train.csv: {train_df.shape}")
            print(f"   🏪 stores.csv: {stores_df.shape}")
            print(f"   📅 features.csv: {features_df.shape}")

            # Log raw dataset shapes
            mlflow.log_metric("raw_train_rows", len(train_df))
            mlflow.log_metric("raw_train_cols", train_df.shape[1])
            mlflow.log_metric("raw_stores_count", len(stores_df))
            mlflow.log_metric("raw_features_rows", len(features_df))

            # Parse dates
            train_df['Date'] = pd.to_datetime(train_df['Date'])
            features_df['Date'] = pd.to_datetime(features_df['Date'])

            # Merge datasets - IMPORTANT: Include IsHoliday in merge keys to avoid duplicates
            # Both train.csv and features.csv have IsHoliday column, so we merge on it
            data = train_df.merge(stores_df, on='Store', how='left')
            data = data.merge(features_df, on=['Store', 'Date', 'IsHoliday'], how='left')

            print(f"📊 Merged dataset shape: {data.shape}")
            mlflow.log_metric("merged_rows", len(data))
            mlflow.log_metric("merged_cols", data.shape[1])

            # Verify IsHoliday column exists
            if 'IsHoliday' not in data.columns:
                raise ValueError("❌ Critical: IsHoliday column not found in merged data!")

            holiday_count = data['IsHoliday'].sum()
            print(f"🎄 Holiday weeks in dataset: {holiday_count}")
            mlflow.log_metric("total_holiday_weeks", int(holiday_count))

            # Remove markdown columns that cause issues
            markdown_cols = [col for col in data.columns if 'markdown' in col.lower()]
            if markdown_cols:
                data = data.drop(columns=markdown_cols)
                print(f"🗑️ Removed markdown columns: {len(markdown_cols)}")
                mlflow.log_metric("markdown_cols_removed", len(markdown_cols))

            # Basic data info
            print(f"📊 Final dataset: {data.shape}")
            print(f"📅 Date range: {data['Date'].min()} to {data['Date'].max()}")
            print(f"🏪 Stores: {data['Store'].nunique()}")
            print(f"🏷️ Departments: {data['Dept'].nunique()}")

            # Log dataset metrics
            mlflow.log_metric("total_records", len(data))
            mlflow.log_metric("stores_count", data['Store'].nunique())
            mlflow.log_metric("departments_count", data['Dept'].nunique())
            mlflow.log_metric("features_count", data.shape[1])
            mlflow.log_param("date_range_start", str(data['Date'].min()))
            mlflow.log_param("date_range_end", str(data['Date'].max()))
            mlflow.log_param("has_isholiday", True)

            # Log data quality metrics
            missing_values = data.isnull().sum().sum()
            mlflow.log_metric("total_missing_values", int(missing_values))
            mlflow.log_metric("missing_percentage", float(missing_values / (len(data) * data.shape[1]) * 100))

            print("✅ Data loading completed!")

        except Exception as e:
            print(f"❌ Error loading data: {str(e)}")
            mlflow.log_param("loading_error", str(e))
            raise

        return data

# ================================================================================
# STEP 3: REUSABLE FEATURE ENGINEERING CLASS
# ================================================================================

class WalmartFeatureEngineer:
    """Reusable feature engineering for Walmart sales data"""

    def __init__(self):
        self.feature_names = []
        self.label_encoders = {}
        self.fitted = False

    def create_date_features(self, df):
        """Create date-based features"""
        df = df.copy()

        # Basic date features
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Quarter'] = df['Date'].dt.quarter
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['Week'] = df['Date'].dt.isocalendar().week
        df['DayOfYear'] = df['Date'].dt.dayofyear

        # Cyclical features (important for seasonality)
        df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
        df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
        df['Quarter_sin'] = np.sin(2 * np.pi * df['Quarter'] / 4)
        df['Quarter_cos'] = np.cos(2 * np.pi * df['Quarter'] / 4)
        df['DayOfWeek_sin'] = np.sin(2 * np.pi * df['DayOfWeek'] / 7)
        df['DayOfWeek_cos'] = np.cos(2 * np.pi * df['DayOfWeek'] / 7)

        # Time progression features
        reference_date = pd.Timestamp('2010-02-05')
        df['DaysFromStart'] = (df['Date'] - reference_date).dt.days
        df['WeeksFromStart'] = df['DaysFromStart'] // 7

        # Holiday-related features
        df['IsHolidayMonth'] = df['Month'].isin([11, 12]).astype(int)
        df['IsQ4'] = (df['Quarter'] == 4).astype(int)

        date_features = ['Year', 'Month', 'Quarter', 'DayOfWeek', 'Week', 'DayOfYear',
                        'Month_sin', 'Month_cos', 'Quarter_sin', 'Quarter_cos',
                        'DayOfWeek_sin', 'DayOfWeek_cos', 'DaysFromStart', 'WeeksFromStart',
                        'IsHolidayMonth', 'IsQ4']

        return df, date_features

    def create_lag_features(self, df, target_col='Weekly_Sales', lags=[1, 2, 4, 8]):
        """Create lag features without data leakage"""
        df = df.copy()
        df = df.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

        lag_features = []
        for lag in lags:
            lag_col = f'{target_col}_lag_{lag}'
            df[lag_col] = df.groupby(['Store', 'Dept'])[target_col].shift(lag)
            lag_features.append(lag_col)

        return df, lag_features

    def create_rolling_features(self, df, target_col='Weekly_Sales', windows=[4, 8, 12]):
        """Create rolling window features"""
        df = df.copy()
        df = df.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

        rolling_features = []
        for window in windows:
            # Rolling statistics with proper index handling
            rolling_group = df.groupby(['Store', 'Dept'])[target_col].rolling(window=window, min_periods=1)

            # Use .values to avoid index conflicts
            df[f'{target_col}_rolling_mean_{window}'] = rolling_group.mean().values
            df[f'{target_col}_rolling_std_{window}'] = rolling_group.std().fillna(0).values
            df[f'{target_col}_rolling_min_{window}'] = rolling_group.min().values
            df[f'{target_col}_rolling_max_{window}'] = rolling_group.max().values

            # Rolling ratios with safe division
            mean_values = df[f'{target_col}_rolling_mean_{window}']
            df[f'{target_col}_vs_rolling_mean_{window}'] = np.where(
                mean_values != 0,
                df[target_col] / mean_values,
                1.0
            )

            rolling_features.extend([
                f'{target_col}_rolling_mean_{window}',
                f'{target_col}_rolling_std_{window}',
                f'{target_col}_rolling_min_{window}',
                f'{target_col}_rolling_max_{window}',
                f'{target_col}_vs_rolling_mean_{window}'
            ])

        return df, rolling_features

    def create_store_dept_features(self, df):
        """Create store and department interaction features"""
        df = df.copy()

        # Store-Department interaction
        df['Store_Dept_Interaction'] = df['Store'].astype(str) + '_' + df['Dept'].astype(str)

        # Store size categories
        df['Store_Size_Category'] = pd.cut(df['Size'], bins=3, labels=['Small', 'Medium', 'Large'])

        store_dept_features = ['Store_Dept_Interaction', 'Store_Size_Category']

        return df, store_dept_features

    def fit_transform(self, train_df):
        """Fit feature engineering on training data and transform"""
        print("🔧 Fitting and transforming data...")

        df = train_df.copy()
        all_features = []

        # 1. Date features
        df, date_features = self.create_date_features(df)
        all_features.extend(date_features)

        # 2. Lag features
        df, lag_features = self.create_lag_features(df)
        all_features.extend(lag_features)

        # 3. Rolling features
        df, rolling_features = self.create_rolling_features(df)
        all_features.extend(rolling_features)

        # 4. Store/Department features
        df, store_dept_features = self.create_store_dept_features(df)
        all_features.extend(store_dept_features)

        # Handle categorical variables
        categorical_cols = ['Type', 'Store_Size_Category', 'Store_Dept_Interaction']
        for col in categorical_cols:
            if col in df.columns:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col].astype(str))
                self.label_encoders[col] = le

        # Fill NaN values
        # For lag and rolling features, forward fill then backward fill
        lag_rolling_cols = lag_features + rolling_features
        for col in lag_rolling_cols:
            if col in df.columns:
                df[col] = df.groupby(['Store', 'Dept'])[col].fillna(method='ffill').fillna(method='bfill')

        # Fill remaining NaNs with 0
        df = df.fillna(0)

        # Store feature names
        self.feature_names = all_features
        self.fitted = True

        print(f"✅ Feature engineering completed!")
        print(f"📊 Total features created: {len(all_features)}")
        print(f"   📅 Date features: {len(date_features)}")
        print(f"   ⏰ Lag features: {len(lag_features)}")
        print(f"   📊 Rolling features: {len(rolling_features)}")
        print(f"   🏪 Store/Dept features: {len(store_dept_features)}")

        return df

    def transform(self, test_df):
        """Transform test/validation data using fitted parameters"""
        if not self.fitted:
            raise ValueError("❌ FeatureEngineer must be fitted before transform!")

        print("🔄 Transforming data...")

        df = test_df.copy()

        # Apply same transformations as training
        df, _ = self.create_date_features(df)
        df, _ = self.create_lag_features(df)
        df, _ = self.create_rolling_features(df)
        df, _ = self.create_store_dept_features(df)

        # Apply saved label encoders
        for col, le in self.label_encoders.items():
            if col in df.columns:
                # Handle unseen categories
                unique_vals = df[col].astype(str).unique()
                for val in unique_vals:
                    if val not in le.classes_:
                        # Add new class to encoder
                        le.classes_ = np.append(le.classes_, val)
                df[col] = le.transform(df[col].astype(str))

        # Fill NaN values same way as training
        lag_rolling_cols = [col for col in df.columns if 'lag_' in col or 'rolling_' in col]
        for col in lag_rolling_cols:
            if col in df.columns:
                df[col] = df.groupby(['Store', 'Dept'])[col].fillna(method='ffill').fillna(method='bfill')

        df = df.fillna(0)

        print("✅ Data transformed!")
        return df

# ================================================================================
# STEP 4: COMPLETE REUSABLE PIPELINE CLASS
# ================================================================================

class WalmartFeaturePipeline:
    """Complete reusable pipeline with save/load functionality"""

    def __init__(self):
        self.feature_engineer = WalmartFeatureEngineer()
        self.fitted = False
        self.feature_columns = None
        self.outlier_bounds = None

    def remove_outliers(self, data):
        """Remove outliers using IQR method"""
        print("🗑️ Removing outliers...")

        # Calculate outlier bounds by Store-Dept groups
        sales_by_group = data.groupby(['Store', 'Dept'])['Weekly_Sales']
        Q1 = sales_by_group.transform(lambda x: x.quantile(0.25))
        Q3 = sales_by_group.transform(lambda x: x.quantile(0.75))
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Store bounds for later use
        self.outlier_bounds = {'lower': lower_bound, 'upper': upper_bound}

        # Identify and remove outliers
        outliers_mask = (data['Weekly_Sales'] < lower_bound) | (data['Weekly_Sales'] > upper_bound)
        outliers_count = outliers_mask.sum()

        data_clean = data[~outliers_mask].copy()

        print(f"🗑️ Removed {outliers_count:,} outliers ({outliers_count/len(data)*100:.1f}%)")
        return data_clean

    def fit(self, data, remove_outliers=True):
        """Fit pipeline on data with outlier removal and feature engineering"""
        with mlflow.start_run(run_name="Pipeline_Fit", nested=True) as run:
            print("🔧 Fitting pipeline...")

            # Log parameters
            mlflow.log_param("remove_outliers", remove_outliers)
            mlflow.log_param("input_data_shape", str(data.shape))
            mlflow.log_metric("input_samples", len(data))

            # Apply outlier removal if requested
            if remove_outliers:
                print("🗑️ Removing outliers...")
                data_clean = self.remove_outliers(data)
                outliers_removed = len(data) - len(data_clean)
                outlier_rate = outliers_removed / len(data) * 100
                print(f"🗑️ Removed {outliers_removed:,} outliers ({outlier_rate:.1f}%)")

                mlflow.log_metric("outliers_removed", outliers_removed)
                mlflow.log_metric("outlier_rate_percent", outlier_rate)
                mlflow.log_metric("clean_samples", len(data_clean))
            else:
                data_clean = data.copy()
                mlflow.log_metric("outliers_removed", 0)
                mlflow.log_metric("outlier_rate_percent", 0.0)

            # Apply feature engineering
            print("🔧 Fitting and transforming data...")
            data_featured = self.feature_engineer.fit_transform(data_clean)

            # Store feature columns (excluding target and metadata)
            exclude_cols = ['Weekly_Sales', 'Date', 'Store_Dept_Interaction', 'IsHoliday']
            self.feature_columns = [col for col in data_featured.columns if col not in exclude_cols]

            print("✅ Feature engineering completed!")
            print(f"📊 Total features created: {len(self.feature_columns)}")

            # Count feature types
            date_features = [col for col in self.feature_columns if any(x in col for x in ['Year', 'Month', 'Quarter', 'Day', 'Week', 'sin', 'cos', 'Holiday', 'FromStart'])]
            lag_features = [col for col in self.feature_columns if 'lag_' in col]
            rolling_features = [col for col in self.feature_columns if any(x in col for x in ['mean_', 'std_', 'min_', 'max_', 'ratio_'])]
            store_dept_features = [col for col in self.feature_columns if any(x in col for x in ['Store', 'Dept', 'Size'])]

            print(f"   📅 Date features: {len(date_features)}")
            print(f"   ⏰ Lag features: {len(lag_features)}")
            print(f"   📊 Rolling features: {len(rolling_features)}")
            print(f"   🏪 Store/Dept features: {len(store_dept_features)}")

            # Log feature counts
            mlflow.log_metric("total_features", len(self.feature_columns))
            mlflow.log_metric("date_features", len(date_features))
            mlflow.log_metric("lag_features", len(lag_features))
            mlflow.log_metric("rolling_features", len(rolling_features))
            mlflow.log_metric("store_dept_features", len(store_dept_features))

            # Log feature names as parameters (first 10 to avoid parameter limit)
            for i, feature in enumerate(self.feature_columns[:10]):
                mlflow.log_param(f"feature_{i+1}", feature)

            # Mark as fitted
            self.fitted = True

            print(f"✅ Pipeline fitted on {len(data_featured):,} samples")
            print(f"📊 Features available: {len(data_featured.columns)}")

            mlflow.log_param("pipeline_fitted", True)
            mlflow.log_metric("final_samples", len(data_featured))
            mlflow.log_metric("final_columns", data_featured.shape[1])

            return data_featured

    def transform(self, data):
        """Transform new data using fitted pipeline"""
        if not self.fitted:
            raise ValueError("❌ Pipeline must be fitted before transform!")

        print("🔄 Transforming new data...")

        # Apply feature engineering
        data_featured = self.feature_engineer.transform(data)

        # Select same features as training (excluding IsHoliday from feature matrix)
        X = data_featured[self.feature_columns].copy()

        # Ensure numeric and fill NaNs
        for col in X.columns:
            if X[col].dtype == 'object':
                X[col] = pd.to_numeric(X[col], errors='coerce')
        X = X.fillna(0)

        print(f"✅ Transformed {len(X):,} samples")
        return X

    def fit_transform(self, data, remove_outliers=True):
        """Fit and transform in one step"""
        data_featured = self.fit(data, remove_outliers)
        return self.transform_fitted_data(data_featured)

    def transform_fitted_data(self, data_featured):
        """Transform already featured data"""
        exclude_cols = ['Weekly_Sales', 'Date', 'Store_Dept_Interaction', 'IsHoliday']
        # Select only the feature columns (exclude target and metadata)
        feature_cols = [col for col in data_featured.columns if col not in exclude_cols]
        X = data_featured[feature_cols].copy()

        for col in X.columns:
            if X[col].dtype == 'object':
                X[col] = pd.to_numeric(X[col], errors='coerce')
        X = X.fillna(0)

        return X

    def save_pipeline(self, filepath='walmart_feature_pipeline.pkl'):
        """Save fitted pipeline"""
        if not self.fitted:
            raise ValueError("❌ Pipeline must be fitted before saving!")

        with open(filepath, 'wb') as f:
            pickle.dump(self, f)

        print(f"💾 Pipeline saved to {filepath}")
        return filepath

    @staticmethod
    def load_pipeline(filepath='walmart_feature_pipeline.pkl'):
        """Load fitted pipeline"""
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"❌ Pipeline file not found: {filepath}")

        with open(filepath, 'rb') as f:
            pipeline = pickle.load(f)

        print(f"📂 Pipeline loaded from {filepath}")
        return pipeline

# ================================================================================
# STEP 5: EVALUATION METRICS
# ================================================================================

def calculate_wmae(y_true, y_pred, is_holiday, holiday_weight=5.0):
    """
    Calculate Weighted Mean Absolute Error (WMAE) for Walmart competition

    Args:
        y_true: Actual sales values
        y_pred: Predicted sales values
        is_holiday: Boolean array indicating holiday weeks
        holiday_weight: Weight multiplier for holiday weeks (default 5.0)

    Returns:
        wmae: Weighted Mean Absolute Error
    """
    abs_errors = np.abs(y_true - y_pred)
    weights = np.where(is_holiday, holiday_weight, 1.0)
    wmae = np.sum(weights * abs_errors) / np.sum(weights)
    return wmae

def evaluate_model_performance(y_train, train_pred, train_is_holiday,
                             y_val, val_pred, val_is_holiday):
    """
    Comprehensive model evaluation with all metrics

    Returns:
        dict: Dictionary containing all evaluation metrics
    """
    # Training metrics
    train_mae = mean_absolute_error(y_train, train_pred)
    train_wmae = calculate_wmae(y_train, train_pred, train_is_holiday)
    train_r2 = r2_score(y_train, train_pred)

    # Validation metrics
    val_mae = mean_absolute_error(y_val, val_pred)
    val_wmae = calculate_wmae(y_val, val_pred, val_is_holiday)
    val_r2 = r2_score(y_val, val_pred)

    metrics = {
        'train_mae': train_mae,
        'train_wmae': train_wmae,
        'train_r2': train_r2,
        'val_mae': val_mae,
        'val_wmae': val_wmae,
        'val_r2': val_r2
    }

    return metrics

# ================================================================================
# STEP 6: USAGE PATTERNS
# ================================================================================

def experiment_pattern():
    """Pattern 1: For experiments with train/val split"""
    with mlflow.start_run(run_name="Experiment_Pattern_Train_Val", nested=True) as run:
        print("🧪 EXPERIMENT PATTERN: Train/Validation Split")

        # Load and split data
        data = load_raw_data()
        train_data, val_data, split_date = create_temporal_split(data)

        # Create pipeline and fit on training only
        pipeline = WalmartFeaturePipeline()
        train_featured = pipeline.fit(train_data, remove_outliers=True)

        # Prepare training data
        X_train = pipeline.transform_fitted_data(train_featured)
        y_train = train_featured['Weekly_Sales']
        train_is_holiday = train_featured['IsHoliday']

        # Transform validation data
        X_val = pipeline.transform(val_data)
        y_val = val_data['Weekly_Sales']
        val_is_holiday = val_data['IsHoliday']

        # Log experiment metrics
        mlflow.log_param("pattern", "experiment")
        mlflow.log_param("split_date", str(split_date))
        mlflow.log_metric("train_samples", len(X_train))
        mlflow.log_metric("val_samples", len(X_val))
        mlflow.log_metric("features_count", X_train.shape[1])

        print(f"📊 Training: {X_train.shape}")
        print(f"📊 Validation: {X_val.shape}")

        return pipeline, X_train, X_val, y_train, y_val, train_is_holiday, val_is_holiday

def create_temporal_split(data, train_ratio=0.8):
    """Create proper temporal split for time series data"""
    with mlflow.start_run(run_name="Temporal_Split_80_20", nested=True) as run:
        print("✂️ Creating temporal split...")

        # Log split parameters
        mlflow.log_param("split_method", "temporal")
        mlflow.log_param("train_ratio", train_ratio)
        mlflow.log_param("validation_ratio", 1 - train_ratio)
        mlflow.log_param("split_type", "time_series_aware")

        # Sort data chronologically
        data_sorted = data.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

        # Calculate split date
        min_date = data_sorted['Date'].min()
        max_date = data_sorted['Date'].max()
        total_days = (max_date - min_date).days
        split_days = int(total_days * train_ratio)
        split_date = min_date + timedelta(days=split_days)

        # Align to Friday (since data is weekly)
        while split_date.weekday() != 4:
            split_date += timedelta(days=1)

        # Create splits
        train_data = data_sorted[data_sorted['Date'] < split_date].copy()
        val_data = data_sorted[data_sorted['Date'] >= split_date].copy()

        # Verify no data leakage
        assert train_data['Date'].max() < val_data['Date'].min(), "❌ DATA LEAKAGE DETECTED!"

        print(f"📅 Split date: {split_date}")
        print(f"🚂 Training: {len(train_data):,} records ({len(train_data)/len(data)*100:.1f}%)")
        print(f"🔮 Validation: {len(val_data):,} records ({len(val_data)/len(data)*100:.1f}%)")
        print(f"📊 Training date range: {train_data['Date'].min()} to {train_data['Date'].max()}")
        print(f"📊 Validation date range: {val_data['Date'].min()} to {val_data['Date'].max()}")

        # Log split metrics
        mlflow.log_param("split_date", str(split_date))
        mlflow.log_metric("train_records", len(train_data))
        mlflow.log_metric("val_records", len(val_data))
        mlflow.log_metric("train_percentage", len(train_data)/len(data)*100)
        mlflow.log_metric("val_percentage", len(val_data)/len(data)*100)
        mlflow.log_param("train_date_start", str(train_data['Date'].min()))
        mlflow.log_param("train_date_end", str(train_data['Date'].max()))
        mlflow.log_param("val_date_start", str(val_data['Date'].min()))
        mlflow.log_param("val_date_end", str(val_data['Date'].max()))

        print("✅ Temporal split completed - no data leakage!")
        return train_data, val_data, split_date

def production_pattern():
    """Pattern 2: For final model on full data"""
    with mlflow.start_run(run_name="Production_Pattern_Full_Data", nested=True) as run:
        print("🚀 PRODUCTION PATTERN: Full Dataset Training")

        # Load full dataset
        data = load_raw_data()

        # Create pipeline and fit on ALL data
        pipeline = WalmartFeaturePipeline()
        full_featured = pipeline.fit(data, remove_outliers=True)

        # Prepare full dataset
        X_full = pipeline.transform_fitted_data(full_featured)
        y_full = full_featured['Weekly_Sales']
        is_holiday_full = full_featured['IsHoliday']

        # Save pipeline for later use
        pipeline_path = pipeline.save_pipeline('walmart_feature_pipeline.pkl')

        # Log production metrics
        mlflow.log_param("pattern", "production")
        mlflow.log_metric("full_samples", len(X_full))
        mlflow.log_metric("features_count", X_full.shape[1])
        mlflow.log_artifact(pipeline_path)

        print(f"📊 Full dataset: {X_full.shape}")
        print(f"💾 Pipeline saved for production use")

        return pipeline, X_full, y_full, is_holiday_full

def testing_pattern(test_data_path=None):
    """Pattern 3: For testing on completely new data"""
    with mlflow.start_run(run_name="Testing_Pattern_New_Data", nested=True) as run:
        print("🔍 TESTING PATTERN: New Data Transformation")

        # Load fitted pipeline
        pipeline = WalmartFeaturePipeline.load_pipeline('walmart_feature_pipeline.pkl')

        # Load new test data (you would provide this)
        if test_data_path:
            test_data = pd.read_csv(test_data_path)
        else:
            print("ℹ️  No test data provided, using sample")
            # For demo, just use part of existing data
            data = load_raw_data()
            test_data = data.sample(100).copy()

        # Transform new data
        X_test = pipeline.transform(test_data)

        # Log testing metrics
        mlflow.log_param("pattern", "testing")
        mlflow.log_metric("test_samples", len(X_test))
        mlflow.log_metric("features_count", X_test.shape[1])

        print(f"📊 Test data: {X_test.shape}")

        return X_test

# ================================================================================
# MAIN EXECUTION PIPELINE
# ================================================================================

def main():
    """Main execution demonstrating complete feature engineering pipeline functionality"""
    print("🚀 EXPERIMENT 5: Complete Feature Engineering Pipeline")
    print("🎯 GOAL: Validate full pipeline functionality with XGBoost")
    print("="*80)

    # Setup
    experiment_name = setup_mlflow()

    print(f"📋 PIPELINE VALIDATION STEPS:")
    print(f"   1. 📂 Load and prepare data")
    print(f"   2. ✂️ Create temporal split (no data leakage)")
    print(f"   3. 🔧 Fit pipeline on training data (with outlier removal)")
    print(f"   4. 🔄 Transform validation data")
    print(f"   5. 🤖 Validate with XGBoost model")
    print(f"   6. 📊 Evaluate WMAE and feature importance")
    print("="*80)

    try:
        with mlflow.start_run(run_name="Complete_Pipeline_Validation") as main_run:
            # Step 1: Load and prepare data
            print(f"\n📂 STEP 1: Loading and preparing data...")
            data = load_raw_data()

            # Step 2: Create temporal split
            print(f"\n✂️ STEP 2: Creating temporal split...")
            train_data, val_data, split_date = create_temporal_split(data)

            # Step 3: Fit pipeline on training data
            print(f"\n🔧 STEP 3: Fitting pipeline on training data...")
            pipeline = WalmartFeaturePipeline()
            train_featured = pipeline.fit(train_data, remove_outliers=True)

            # Prepare training data (IsHoliday is preserved in train_featured after outlier removal)
            X_train = pipeline.transform_fitted_data(train_featured)
            y_train = train_featured['Weekly_Sales']
            train_is_holiday = train_featured['IsHoliday'].values  # Use IsHoliday from the featured data

            # Step 4: Transform validation data
            print(f"\n🔄 STEP 4: Transforming validation data...")
            X_val = pipeline.transform(val_data)
            y_val = val_data['Weekly_Sales']
            val_is_holiday = val_data['IsHoliday'].values  # Use IsHoliday from original val_data

            # Log pipeline metrics
            mlflow.log_param("pipeline_type", "complete_validation")
            mlflow.log_param("split_date", str(split_date))
            mlflow.log_metric("train_samples", len(X_train))
            mlflow.log_metric("val_samples", len(X_val))
            mlflow.log_metric("features_count", X_train.shape[1])
            mlflow.log_metric("stores_count", data['Store'].nunique())
            mlflow.log_metric("departments_count", data['Dept'].nunique())
            mlflow.log_metric("train_holiday_weeks", int(train_is_holiday.sum()))
            mlflow.log_metric("val_holiday_weeks", int(val_is_holiday.sum()))

            print(f"\n📊 PIPELINE SUMMARY:")
            print(f"   🚂 Training data: {X_train.shape}")
            print(f"   🔮 Validation data: {X_val.shape}")
            print(f"   📈 Features created: {X_train.shape[1]}")
            print(f"   📅 Split date: {split_date}")
            print(f"   🎄 Training holiday weeks: {train_is_holiday.sum()}")
            print(f"   🎄 Validation holiday weeks: {val_is_holiday.sum()}")

            # Verify data consistency
            print(f"\n🔍 DATA CONSISTENCY CHECKS:")
            print(f"   ✅ Train X shape matches y: {len(X_train) == len(y_train)}")
            print(f"   ✅ Train IsHoliday matches: {len(train_is_holiday) == len(y_train)}")
            print(f"   ✅ Val X shape matches y: {len(X_val) == len(y_val)}")
            print(f"   ✅ Val IsHoliday matches: {len(val_is_holiday) == len(y_val)}")

            # Step 5 & 6: Validate with XGBoost and evaluate
            print(f"\n🤖 STEP 5-6: Final XGBoost validation...")
            final_metrics = validate_complete_pipeline_with_xgboost(
                X_train, X_val, y_train, y_val, train_is_holiday, val_is_holiday
            )

            # Log final results
            for metric_name, value in final_metrics.items():
                mlflow.log_metric(f"final_{metric_name}", value)

            # Log additional useful metrics
            mlflow.log_metric("data_leakage_check", 1.0)  # Passed if we get here
            mlflow.log_metric("feature_engineering_success", 1.0)
            mlflow.log_metric("outlier_removal_rate", (len(train_data) - len(train_featured)) / len(train_data))

            print(f"\n" + "="*80)
            print(f"🎉 COMPLETE PIPELINE VALIDATION SUCCESSFUL!")
            print(f"🎯 Final Validation WMAE: ${final_metrics['val_wmae']:,.2f}")
            print(f"📊 Features: {X_train.shape[1]}")
            print(f"🚂 Training samples: {X_train.shape[0]:,}")
            print(f"🔮 Validation samples: {X_val.shape[0]:,}")
            print(f"📈 Validation R²: {final_metrics['val_r2']:.4f}")
            print(f"🎄 Holiday weeks processed: {train_is_holiday.sum() + val_is_holiday.sum()}")
            print(f"💾 Pipeline ready for production use!")
            print(f"="*80)

            # Optional: Show how to use in production
            print(f"\n💡 PRODUCTION USAGE:")
            print(f"   1. Call production_pattern() to fit on full dataset")
            print(f"   2. Save pipeline with pipeline.save_pipeline()")
            print(f"   3. Load with WalmartFeaturePipeline.load_pipeline()")
            print(f"   4. Transform new data with pipeline.transform()")
            print(f"   5. Keep IsHoliday column separate for WMAE calculation")

    except Exception as e:
        print(f"❌ Error in pipeline validation: {str(e)}")
        import traceback
        traceback.print_exc()
        raise

    return final_metrics

def validate_complete_pipeline_with_xgboost(X_train, X_val, y_train, y_val, train_is_holiday, val_is_holiday):
    """Complete pipeline validation using XGBoost with comprehensive evaluation"""
    with mlflow.start_run(run_name="XGBoost_Pipeline_Validation", nested=True) as run:
        print("🤖 Validating complete pipeline with XGBoost...")

        # XGBoost parameters optimized for Walmart sales
        xgb_params = {
            'n_estimators': 500,
            'max_depth': 8,
            'learning_rate': 0.1,
            'subsample': 0.85,
            'colsample_bytree': 0.85,
            'reg_alpha': 1.0,
            'reg_lambda': 1.5,
            'random_state': 42,
            'n_jobs': -1,
            'objective': 'reg:squarederror'
        }

        # Train XGBoost model
        print("   🏋️ Training XGBoost model...")
        model = xgb.XGBRegressor(**xgb_params)
        model.fit(X_train, y_train, verbose=False)

        # Make predictions
        print("   🔮 Making predictions...")
        train_pred = model.predict(X_train)
        val_pred = model.predict(X_val)

        # Comprehensive evaluation
        print("   📊 Evaluating performance...")
        metrics = evaluate_model_performance(
            y_train, train_pred, train_is_holiday,
            y_val, val_pred, val_is_holiday
        )

        # Feature importance analysis
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)

        # Display comprehensive results
        print(f"\n" + "="*60)
        print(f"🎯 COMPLETE PIPELINE VALIDATION RESULTS")
        print(f"="*60)

        print(f"\n🚂 TRAINING PERFORMANCE:")
        print(f"   WMAE: ${metrics['train_wmae']:,.2f}")
        print(f"   MAE:  ${metrics['train_mae']:,.2f}")
        print(f"   R²:   {metrics['train_r2']:.4f}")

        print(f"\n🔮 VALIDATION PERFORMANCE:")
        print(f"   WMAE: ${metrics['val_wmae']:,.2f}")
        print(f"   MAE:  ${metrics['val_mae']:,.2f}")
        print(f"   R²:   {metrics['val_r2']:.4f}")

        print(f"\n🎯 TOP 15 MOST IMPORTANT FEATURES:")
        for i, (_, row) in enumerate(feature_importance.head(15).iterrows()):
            print(f"   {i+1:2d}. {row['feature']:30s}: {row['importance']:.4f}")

        # Performance assessment
        if metrics['val_wmae'] < 5000:
            performance = "🟢 EXCELLENT"
        elif metrics['val_wmae'] < 10000:
            performance = "🟡 GOOD"
        else:
            performance = "🔴 NEEDS IMPROVEMENT"

        print(f"\n📈 PIPELINE ASSESSMENT: {performance}")
        print(f"   Validation WMAE: ${metrics['val_wmae']:,.2f}")

        # Log all metrics and parameters
        for metric_name, value in metrics.items():
            mlflow.log_metric(metric_name, value)

        for param_name, value in xgb_params.items():
            mlflow.log_param(f"xgb_{param_name}", value)

        mlflow.log_param("model_type", "XGBRegressor")
        mlflow.log_param("validation_type", "complete_pipeline")

        # Log feature importance
        mlflow.log_metric("top_feature_importance", feature_importance.iloc[0]['importance'])
        mlflow.log_param("most_important_feature", feature_importance.iloc[0]['feature'])

        # Log model artifact
        mlflow.sklearn.log_model(
            model,
            "xgboost_model",
            registered_model_name="Walmart_Sales_XGBoost_Pipeline"
        )

        # Log feature importance as artifact
        feature_importance_path = "feature_importance.csv"
        feature_importance.to_csv(feature_importance_path, index=False)
        mlflow.log_artifact(feature_importance_path)

        # Log additional model metrics
        mlflow.log_metric("num_features", len(X_train.columns))
        mlflow.log_metric("train_val_wmae_ratio", metrics['train_wmae'] / metrics['val_wmae'])
        mlflow.log_metric("train_samples", len(X_train))
        mlflow.log_metric("val_samples", len(X_val))

        print(f"\n✅ Complete pipeline validation finished!")
        print(f"📊 All metrics logged to MLflow")
        print(f"🤖 Model saved as artifact")
        print(f"📈 Feature importance saved as CSV")
        print(f"="*60)

        return metrics

if __name__ == "__main__":
    results = main()


🚀 EXPERIMENT 5: Complete Feature Engineering Pipeline
🎯 GOAL: Validate full pipeline functionality with XGBoost
🔧 Setting up MLflow and DagsHub...


Initialized MLflow to track repo "konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting"

Repository konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting initialized!

✅ DagsHub initialized successfully!
✅ Created new experiment: Experiment_5_Feature_Engineering_Pipeline_20250703_102701
✅ MLflow setup complete!
🔗 Tracking URI: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow
📊 Experiment: Experiment_5_Feature_Engineering_Pipeline_20250703_102701
📋 PIPELINE VALIDATION STEPS:
   1. 📂 Load and prepare data
   2. ✂️ Create temporal split (no data leakage)
   3. 🔧 Fit pipeline on training data (with outlier removal)
   4. 🔄 Transform validation data
   5. 🤖 Validate with XGBoost model
   6. 📊 Evaluate WMAE and feature importance

📂 STEP 1: Loading and preparing data...
📂 Loading datasets...
✅ Loaded datasets:
   📊 train.csv: (421570, 5)
   🏪 stores.csv: (45, 3)
   📅 features.csv: (8190, 12)
📊 Merged dataset shape: (421570, 16)
🎄 Holiday weeks in dataset: 29661
🗑️ Removed markdown columns: 5
📊 Final dataset: (421570, 11)
📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🏪 Stores: 45
🏷️ Departments: 81
✅ Data loa

2025/07/03 10:30:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBoost_Pipeline_Validation at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/14/runs/d1e84ea2b1784c748e266081ce44581b
🧪 View experiment at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/14
🏃 View run Complete_Pipeline_Validation at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/14/runs/0a13de54f09d4c12b728c09cd14b6330
🧪 View experiment at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/14
❌ Error in pipeline validation: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}


Traceback (most recent call last):
  File "/tmp/ipython-input-64-2955994909.py", line 789, in main
    final_metrics = validate_complete_pipeline_with_xgboost(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-64-2955994909.py", line 916, in validate_complete_pipeline_with_xgboost
    mlflow.sklearn.log_model(
  File "/usr/local/lib/python3.11/dist-packages/mlflow/sklearn/__init__.py", line 426, in log_model
    return Model.log(
           ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/mlflow/models/model.py", line 1161, in log
    model = mlflow.initialize_logged_model(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/mlflow/tracking/fluent.py", line 2130, in initialize_logged_model
    model = _create_logged_model(
            ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/mlflow/tracking/fluent.py", line 2257, in _create_logged_model
    return MlflowClient().create_l

RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}